In [1]:
import requests
from bs4 import BeautifulSoup
from random import uniform
import csv
import time
import csv

In [2]:
bookId = '8595084742'
url = 'https://www.amazon.com.br/Hobbit-p%C3%B4ster-J-R-R-Tolkien/product-reviews/' + bookId + '/ref=cm_cr_dp_d_show_all_btm?ie=UTF8&reviewerType=all_reviews&pageNumber='
# If it reaches a page that doesn't exist, it stops looking for more pages 
end = False
x = 1

In [3]:
# Open the csv file where the information will be stored
with open('datasets/csv_file_NEW.csv', 'w', encoding='UTF8', newline='') as f:
    writer = csv.writer(f, delimiter=';', quoting=csv.QUOTE_MINIMAL)

    # [Daniel]
    # I put this line to easy the read fo the file on the notebook analysis.ipynb.
    # Without this line, the first review become the head of the file when read in the pandas.
    writer.writerow( ['bookId;reviewId;reviewText;'] )
    
    # While it finds reviews
    while not end:
        
        ######## ONLY FIRST 10 PAGES (for testing purposes)
        # end = True
        
        # Reads only 10 pages and the sleeps for 3 seconds, trying to avoid 503 back from Amazon
        for pageNumber in range(x, x + 10):    
            
            # Get the next page of the review
            page = requests.get(url + str(pageNumber))

            # If it gets a 503 error from Amazon, sleeps for some seconds and try again until it works
            while page.status_code == 503:
                time.sleep(uniform(1.0, 5.0))
                page = requests.get(url + str(pageNumber))
            
            # Initializing the beautifulsoup for manipulation the html text
            soup = BeautifulSoup(page.text, 'html.parser')
            # Selecting the reviews from the page
            reviews = soup.select('.review')

            # If it doesn't find any reviews, it has reached the last review and might finish
            if not reviews:
                end = True
                break;
            
            print('**** READING PAGE ' + str(pageNumber) + ' *********************************')
            
            # For each review found in the page
            for review in reviews:
                reviewId = review.get('id')

                reviewTextContent = review.select('.review-text-content')
                spamQuantity = len(reviewTextContent[0].select('span'))

                # If the review text is not empty, try to sanitize the text
                if spamQuantity != 0:
                    text = reviewTextContent[0].select('span')[spamQuantity - 1]
                    text = str(text).replace(';', ',')
                    text = str(text).replace('"', '')
                    text = str(text).replace('<span>', '')
                    text = str(text).replace('</span>', '')
                    text = str(text).replace('<br/>', '')
                    text = str(text).replace('</br>', '')
                    text = str(text).replace('<br>', '')
                    text = text.strip()
                    print(text)
                    writer.writerow([bookId + ';' + reviewId + ';' + text + ';'])
                    print('===================================================')
        # Advance to the next page that hasn't been reached yet
        x = int(x) + 10  
        # Sleeps for 3 seconds, trying to avoid 503 back from Amazon 
        time.sleep(3)


**** READING PAGE 1 *********************************
O livro, que já havia lido há tempos e quero renovar a leitura, é simplesmente excelente e maravilhoso. A edição e encadernamento da editora são encantadoras, a capa é dura e totalmente ilustrada. O pôster é lindo e inclusive irei emoldurar :)Enfim, o produto veio em perfeito estado e é altamente recomendado por mim.Além disso, a entrega foi extremamente rápida, através da Total Express, e chegou antes do prazo (que já é enxuto). Eu fiz o pedido no dia 06/11 e chegou em minha residência no dia 11/11 (teve final de semana nesse período, sendo que o produto chegou em minha cidade no dia 09/11).Finalmente, tanto o produto quanto a entrega são perfeitos! Simplesmente isso.
Somos capazes de sentir o cuidado e a paixão com que um produto foi feito ao colocar-mos as mãos nele. Esta edição da HarperCollins é este tipo de produto, aliás, todas as edições referentes a Tolkien publicados pela editora até o momento são este tipo de produto.A ca

KeyboardInterrupt: 